In [ ]:
!pip install gradio transformers

In [ ]:
import gradio as gr
from transformers import pipeline
import requests
import urllib.request
import numpy as np
import nltk
nltk.download('punkt')
from nltk import tokenize
from PIL import Image, ImageDraw, ImageFont


generator = pipeline('text-generation', model='gpt2-large')
url = "https://api.newnative.ai/stable-diffusion?prompt=A digital illustration of story with fantasy and magic. "
allStory = []
story = ""
allStory=""

def generateStory(generator, url, story, allStory):
    story = allStory
    if story != "":
        story = story
        
    if (allStory == ""):
        allStory = generator("Once upon a time there was a ")
    else:
        allStory = generator(allStory) 

    allStory = allStory[0]['generated_text']

    return story, allStory


def generateImage(url, story, allStory):
    story, allStory = generateStory(generator, url, story, allStory)
    a = tokenize.sent_tokenize(allStory)
    b = tokenize.sent_tokenize(story)
    text = np.setdiff1d(a, b)
    text = text.tolist()
    text = ' '.join(text[::-1])

    url = url + text
    response = requests.request("GET", url)
    data = response.json()
    imgUrl = data["image_url"]
    img_file = urllib.request.urlretrieve(data["image_url"], "/content/local-filename.jpg")


#    img, imgfile = generateImage(url, story, allStory)
    img = generateImage(url, story, allStory)

    image = Image.open(img)
    W, H = 512, 512
    draw = ImageDraw.Draw(image)
    _, _, w, h = draw.textbbox((0, 0), allStory,)
    draw.text(((W-w)/2, (H-h)/2), allStory)

    return draw

# create the gradio UI
# set precision to 0 to round value to nearest int
demo = gr.Interface(
    fn=generateImage,
    inputs=[url, story, allStory],
    outputs = "image"
    )

# allow queueing or incoming requests, max=3 
#demo.queue(concurrency_count=3)
#demo = gr.Interface(generateStory(generator, url, story, allStory), gr.Image(shape=(512, 512)), "image")

# launch demo
demo.launch(debug=True)